# Convert — Run a simple conversion of a PDB to the QDXF format

This notebook shows how to perform a simple conversion of PDBs to the QDXF format

# 0) Complete example
See the [sample notebook](/Quickstarts/convert-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

In [ ]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
    restore_by_default=True
)

# 2) Fetch PDBS

In [ ]:
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)

# 3) Convert each PDB

In [ ]:
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = await client.convert("PDB", Path(pdb), target="NIX")
    print(await conformer.get())

2024-02-25 16:59:07,537 - rush - INFO - Trying to restore job with tags: ['qdx', 'rush-py-v2', 'demo', 'convert'] and path: github:talo/tengu-prelude/6ce15cd56dc7a3cec8e5e6951bb473e33c5ecd11#convert
2024-02-25 16:59:07,845 - rush - INFO - Argument 62e0450a-1e52-4969-ad70-0734270be792 is now ModuleInstanceStatus.RESOLVING
2024-02-25 16:59:08,925 - rush - INFO - Argument 62e0450a-1e52-4969-ad70-0734270be792 is now ModuleInstanceStatus.ADMITTED
2024-02-25 16:59:51,161 - rush - INFO - Argument 62e0450a-1e52-4969-ad70-0734270be792 is now ModuleInstanceStatus.DISPATCHED
2024-02-25 16:59:52,241 - rush - INFO - Argument 62e0450a-1e52-4969-ad70-0734270be792 is now ModuleInstanceStatus.QUEUED
2024-02-25 16:59:53,349 - rush - INFO - Argument 62e0450a-1e52-4969-ad70-0734270be792 is now ModuleInstanceStatus.AWAITING_UPLOAD
{'url': 'https://storage.googleapis.com/qdx-store/d82e623c-7483-4c66-9b98-69ddc91d66e6?x-goog-signature=1ce86ac00268d615826fba94766389e07a1efe0271bdf2bdfa93dfe66f5b7f8e79260a3dd4

In [ ]:
await client.status()

{'1cb77943-9d46-4f9f-ac66-1a9d9759db7d': (<ModuleInstanceStatus.COMPLETED: 'COMPLETED'>,
  'convert',
  1)}